# Image Stack Viewer Workflow

<div style="text-align: center;">
    <img src="./assets/230620_video-viewer.png" alt="video viewer preview" width="450"/>
</div>

## Summary
This workflow is dedicated to visualizing Miniscope calcium imaging data. It's designed for handling large datasets typical in neuroscience, especially single-photon miniscope data, providing an efficient visualization tool with features tailored to the needs of this field.

## Setup and Imports

<div class="admonition alert alert-info">
<p><strong> Dependencies</strong></p>

This workflow notebook requires the packages specified in the [environment file](./environment.yml) in this workflow directory.
</div>


In [ ]:
import os
import sys
from functools import partial

import numpy as np
import xarray as xr
import panel as pn
import holoviews as hv

from neurodatagen.ca_imaging import simulate_miniscope_data
from holoviews.streams import Stream
from hvplot import xarray

# Configuration
%load_ext autoreload
%autoreload 2

# Extensions
pn.extension(throttled=False)
hv.extension('bokeh')

## Data Handling Overview

Effective data handling is crucial for neuroscience imaging. This section outlines our approach to managing both synthetic and real data, highlighting the key differences and our strategies for processing them.


### Generating Synthetic Data

Synthetic data generation is an essential step for testing, developing, and demonstrating visualization tools. We use simulated data to mimic the properties of real miniscope data, ensuring our tools are reliable and accurate.

The `simulate_miniscope_data` function used below generates synthetic calcium imaging data by simulating neural activity through spatial footprints and temporal calcium traces, while incorporating motion artifacts and realistic background noise. The function outputs a numpy array of 8-bit unsigned integers, encapsulated within an `xarray.DataArray`. This array represents the generated imaging data with dimensions corresponding to frame height, frame width, and frame number.

In [ ]:
# Constants
NCELL = 50
DIMS = {'height': 512, 'width': 512, 'frame': 400}
ARR_NAME = 'sim_miniscope'
CHK_SIZE = 200  # Chunk size of frames
CHUNKS = {'frame': CHK_SIZE}  # Chunk by set of complete frames

# Simulate miniscope data
sim_data = simulate_miniscope_data(ncell=NCELL, dims=DIMS, arr_name=ARR_NAME, chk_size=CHK_SIZE)

# Display data
sim_data

# TODO: Set chunks here
# TODO: Add option to generate data straight to disk

#### Writing Data to Disk
In neuroscience, it's very common to encounter datasets too large to fit entirely in memory. To emulate this, we write our simulated dataset to disk (even though it's not so big), adopting a strategy that allows for memory-efficient reading. This approach is crucial for managing large datasets without overwhelming your system's memory. The 'Bytes' information in the xarray output provides a useful indicator of the dataset's size, helping you to plan your data handling strategy.

<div class="admonition alert alert-success">
<p><strong> Tip: </strong></p>

Toggle the `WRITE_SIM`, `READ_SIM` flags below to `False` to bypass data writing/reading during repeated experimentation.
</div>

In [ ]:
# Constants
WRITE_SIM = True  # Flag to enable data writing to disk
SIM_DPATH = '~/data/miniscope_simulated.zarr'  # Location to be stored to or loaded from

# If the WRITE flag is set to True, store the data in Zarr format, suitable for large datasets
if WRITE_SIM:
    sim_data.to_zarr(SIM_DPATH, mode='w', consolidated=True)
    del sim_data  # Delete the data from memory to validate reading from disk in the next step

#### Reading Data from Disk
Let's read the data, emulating the situation where a dataset is too large to fit into memory.

In [ ]:
# Control flag for reading the simulated data from disk
READ_SIM = True

if READ_SIM:
    # Open the dataset from the Zarr storage
    ldataset = xr.open_dataset(SIM_DPATH, engine='zarr', chunks=CHUNKS)
    
    # Access the specific data array of interest
    data = ldataset[ARR_NAME]

<div class="admonition alert alert-info">
<p><strong> Note: </strong></p>

Utilizing the `chunks` parameter in `xr.open_dataset` above is crucial for efficient data handling with large datasets, as it enables Dask to process the data in manageable portions.
</div>

### Working with Real Data

Rea data always introduces unique challenges due to idiosyncracies of each experimental data acquisition approach. Here we detail how we adapt our workflow to efficiently process and prepare one example of real data for visualization.


In [ ]:
# TODO: host and show access to the real demo dataset, sourced from the Minian repo.
# The conversion from raw .avi files to the xarray/zarr format using Minian is in 231218_backup_workflow_image-stack.ipynb
# Update this when Minian devs make a conda release for osx_arm64

## Visualization Strategies

Visualizing calcium imaging data effectively is key to extracting meaningful insights. We introduce various visualization approaches to cater to different analysis needs.

We will start with one-liner viewer and then proceed to a more advanced application with enhanced interactive features and exposed controls, at the expense of code complexity.

### Version 1: Basic Viewer

The Basic Viewer provides a one-liner fundamental visualization interface. It's designed to offer a straightforward and clear view of imaging data, suitable for initial analyses and quick inspections.

#### Synthetic data

In [ ]:
data.hvplot.image(groupby="frame", cmap="Viridis", height=400, width=400, colorbar=False)

#### Real data

In [ ]:
# TODO: Waiting on Minian to make a conda release for osx_arm64 so we can use it to show the conversion of .avi to xarray/zarr without having to rely on cloning a fork
# real_data.hvplot.image(groupby="frame", cmap="Viridis", height=400, width=400, colorbar=False)

### Version 2: Advanced Viewer

The Advanced Viewer builds on the Basic Viewer with added functionality:

1. **Video Playback:** Features a video player widget for continuous playback, along with controls for step-by-step examination of the image stack.

2. TODO: **Interactive Annotations** Enables direct annotation of regions of interest within the visual interface utilizing the HoloNote package. These are linked to timeseries plots for the annotated regions.

3. **Synchronized Side Views** Provides aggregated side views synchronized with the playback of the main image stack view, facilitating examination across different dimensions and aiding in identifying temporal or spatial patterns.

4. TODO **Data Intensity and Statistics** Integrates an intensity histogram for levels adjustment, alongside summary statistics that offer a quick quantitative overview of fluorescence signals.

5. **Customization Capabilities:** Designed for adaptability, allowing users to tailor visualization settings for their specific data, and extensible to accommodate future developments in imaging analysis.

#### Synthetic data [Advanced]

In [ ]:
# Constants
FRAMES_PER_SECOND = 30
FRAMES = data.coords["frame"].values

# Create a video player widget
video_player = pn.widgets.Player(
    length=len(data.coords["frame"]),
    interval=1000 // FRAMES_PER_SECOND,  # Milliseconds
    value=int(FRAMES.min()),
    max_width=400,
    max_height=90,
    loop_policy="loop",
    sizing_mode="stretch_width",
)

# Create a main plot
main_plot = data.hvplot.image(
    groupby="frame",
    cmap="Viridis",
    frame_height=400,
    frame_width=400,
    colorbar=False,
    widgets={"frame": video_player},
)

# frame indicator lines on side plots
line_opts = dict(color='red', alpha=.6, line_width=3)
dmap_hline = hv.DynamicMap(pn.bind(lambda value: hv.HLine(value), video_player)).opts(**line_opts)
dmap_vline = hv.DynamicMap(pn.bind(lambda value: hv.VLine(value), video_player)).opts(**line_opts)

right_plot = data.max(['width']).hvplot.image(x='frame',
    cmap="Viridis",
    frame_height=400,
    frame_width=200,
    colorbar=False,
    title='_', # TODO: Fix this. See https://github.com/bokeh/bokeh/issues/13225#issuecomment-1611172355
) * dmap_vline

# TODO: Have frame progression going from main plot
bottom_plot = data.max(['height']).hvplot.image(y='frame',
    cmap="Viridis",
    frame_height=200,
    frame_width=400,
    colorbar=False,
    # invert_yaxis=True # TODO: Fix this. See https://github.com/holoviz/holoviews/issues/5801
    # ylim = (data.frame.size-1,0), # TODO: OR Fix this. Supposed to be a invert_yaxis workaround. See https://discourse.holoviz.org/t/hvplot-image-invert-y-axis/5625/2?u=droumis
) * dmap_hline

video_player.margin = (20, 20, 20, 70) # TODO: Fix this. Hack to center widget over main

# Below, select just main_plot[0] so we can handle the widget placement explicitly
sim_app = pn.Column(
    video_player,
    pn.Row(main_plot[0], right_plot),
    bottom_plot)

sim_app

#### Real data [Advanced]

In [ ]:
# TODO: Waiting on Minian to make a conda release for osx_arm64 so we can use it to show the conversion of .avi to xarray/zarr without having to rely on cloning a fork
# The viz of raw data is in 231218_backup_workflow_image-stack.ipynb

In [ ]:
# TODO: Consider including additional advanced versions are in 231218_backup_workflow_image-stack.ipynb